Copyright (c) 2023, Troy Phat Tran (Mr. Troy).

Question:

Build and train a binary classifier for the language classification dataset. The dataset is typically a JSON array<br>
of 500 JSON objects. Each object has 3 keys: sentence, language_code, and is_english.<br>
We want our model to be able to determine whether a piece of text is "English or not".

Your task is to fill in the missing parts of the code block (where commented as "ADD CODE HERE").

Note: the dataset is imbalanced as there are more non-English sentences than English ones. To keep things simple, <br>
you don't need to handle data imbalance in this coding challenge.

In [1]:
import json
import os
from urllib.request import urlretrieve

In [3]:
import numpy as np
from keras import Sequential
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import tensorflow as tf

In [4]:
def nlp_binary_model():
    # Download the dataset
    json_file = 'language-classification.json'
    if not os.path.exists(json_file):
        url = 'https://trientran.github.io/tf-practice-exams/language-classification.json'
        urlretrieve(url=url, filename=json_file)

    # Parse the JSON file
    with open(file=json_file, mode='r', encoding='utf-8') as f:
        datastore = json.load(f)

    # Extract texts and labels from JSON data
    texts = []
    labels = []
    for item in datastore:
        texts.append(item['sentence'])  # replace with the sentence/paragraph/text field in the real test JSON file
        labels.append(item['is_english'])  # replace with the label field in the real test JSON file

    # Predefined constants
    max_length = 25
    trunc_type = 'pre'  # Can be replaced with 'post'
    vocab_size = 500
    padding_type = 'pre'  # Can be replaced with 'post'
    embedding_dim = 32
    oov_tok = "<OOV>"

    # Split the dataset into training and validation sets
    num_samples = len(texts)
    num_train_samples = int(0.8 * num_samples)
    indices = np.random.permutation(num_samples)
    train_indices = indices[:num_train_samples]
    val_indices = indices[num_train_samples:]

    # Tokenize the texts
    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    # Pad the sequences
    padded_sequences = pad_sequences(
        sequences=sequences,
        maxlen=max_length,
        padding=padding_type,
        truncating=trunc_type
    )
    padded_training_set = padded_sequences[train_indices]
    padded_validation_set = padded_sequences[val_indices]

    # Convert the labels to numpy array
    training_labels = np.array(labels)[train_indices]
    validation_labels = np.array(labels)[val_indices]

    # Define the model architecture
    model = Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Define an early stopping callback (optional)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

    # Train the model
    model.fit(padded_training_set, training_labels, validation_data=(padded_validation_set, validation_labels), epochs=50, callbacks=[early_stopping])
    return model

===============DO NOT EDIT THIS PART================================

In [5]:
if __name__ == '__main__':
    # Run and save your model
    my_model = nlp_binary_model()
    filepath = "nlp_binary_model.h5"
    my_model.save(filepath)

    # Reload the saved model
    saved_model = load_model(filepath)

    # Show the model architecture
    saved_model.summary()

Epoch 1/50
13/13 [==============================] - 18s 654ms/step - loss: 0.6608 - accuracy: 0.7425 - val_loss: 0.6084 - val_accuracy: 0.8200
Epoch 2/50
13/13 [==============================] - 2s 117ms/step - loss: 0.5499 - accuracy: 0.7950 - val_loss: 0.4446 - val_accuracy: 0.8200
Epoch 3/50
13/13 [==============================] - 2s 117ms/step - loss: 0.4848 - accuracy: 0.7950 - val_loss: 0.4269 - val_accuracy: 0.8200
Epoch 4/50
13/13 [==============================] - 1s 117ms/step - loss: 0.4264 - accuracy: 0.7950 - val_loss: 0.3433 - val_accuracy: 0.8200
Epoch 5/50
13/13 [==============================] - 1s 56ms/step - loss: 0.3056 - accuracy: 0.8075 - val_loss: 0.1997 - val_accuracy: 0.8900
Epoch 6/50
13/13 [==============================] - 1s 70ms/step - loss: 0.1502 - accuracy: 0.9500 - val_loss: 0.0808 - val_accuracy: 0.9900
Epoch 7/50
13/13 [==============================] - 1s 54ms/step - loss: 0.0549 - accuracy: 0.9975 - val_loss: 0.0179 - val_accuracy: 1.0000
Epoch 8/

/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 32)            16000     
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 24)                1560      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 34225 (133.69 KB)
Trainable params: 34225 (133.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
